In [2]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import pickle

In [3]:
loan_data_inputs_train=pd.read_csv('Data/loan_data_inputs_train.csv',index_col=0)
loan_data_targets_train=pd.read_csv('Data/loan_data_targets_train.csv',index_col=0,header=None)
loan_data_inputs_test=pd.read_csv('Data/loan_data_inputs_test.csv',index_col=0)
loan_data_targets_test=pd.read_csv('Data/loan_data_targets_test.csv',index_col=0,header=None)

C:\Users\HP\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# Explore Data

In [4]:
loan_data_targets_train=pd.Series(loan_data_targets_train.iloc[1:,0])

In [5]:
loan_data_targets_train.index=loan_data_targets_train.index.astype('int64')

In [6]:
loan_data_targets_train

0
327896    1
237691    1
249332    1
88093     0
217265    0
         ..
136366    1
270195    0
131746    1
378475    1
171250    1
Name: 1, Length: 373028, dtype: object

In [7]:
loan_data_targets_test=pd.Series(loan_data_targets_test.iloc[1:,0])

In [8]:
loan_data_targets_test.index=loan_data_targets_test.index.astype('int64')

In [9]:
loan_data_targets_test

0
274292    1
346131    1
465415    1
98912     1
452655    1
         ..
91579     1
231699    1
26196     1
319215    1
174927    0
Name: 1, Length: 93257, dtype: object

In [10]:
(loan_data_targets_test.values=='good_bad').sum()

0

In [11]:
loan_data_inputs_test

,Unnamed: 0.1,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,...,dti:21.7-22.4,dti:22.4-35,dti:>35,mths_since_last_record:Missing,mths_since_last_record:0-2,mths_since_last_record:3-20,mths_since_last_record:21-31,mths_since_last_record:32-80,mths_since_last_record:81-86,mths_since_last_record:>86
274292,274292,31708045,34311246,8125,8125,8125.000000,36,14.99,281.62,C,...,0,0,0,1,0,0,0,0,0,0
346131,346131,21179763,23482675,3825,3825,3825.000000,36,15.61,133.75,D,...,0,0,0,1,0,0,0,0,0,0
465415,465415,10079489,11931112,10000,10000,10000.000000,36,13.98,341.68,C,...,0,0,0,1,0,0,0,0,0,0
98912,98912,6914877,8556986,20000,20000,20000.000000,36,6.62,614.08,A,...,0,0,0,1,0,0,0,0,0,0
452655,452655,11065117,12977270,5000,5000,5000.000000,36,23.40,194.60,E,...,0,1,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91579,91579,6920128,7431439,9600,9600,9600.000000,36,17.56,344.95,D,...,0,0,0,1,0,0,0,0,0,0
231699,231699,37621093,40383982,20000,20000,20000.000000,36,6.99,617.46,A,...,0,0,0,1,0,0,0,0,0,0
26196,26196,573638,737847,10000,10000,9994.815151,36,15.58,349.51,D,...,0,0,0,1,0,0,0,0,0,0
319215,319215,23483271,25836184,12000,12000,12000.000000,60,14.49,282.28,C,...,0,1,0,1,0,0,0,0,0,0


In [12]:
loan_data_inputs_train

,Unnamed: 0.1,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,...,dti:21.7-22.4,dti:22.4-35,dti:>35,mths_since_last_record:Missing,mths_since_last_record:0-2,mths_since_last_record:3-20,mths_since_last_record:21-31,mths_since_last_record:32-80,mths_since_last_record:81-86,mths_since_last_record:>86
327896,327896,23964931,26337651,12000,12000,12000.0,36,16.29,423.61,D,...,0,0,0,0,0,0,0,0,1,0
237691,237691,36261200,38972610,19800,19800,19800.0,60,17.86,501.29,D,...,0,0,0,1,0,0,0,0,0,0
249332,249332,34884391,37567722,2000,2000,2000.0,36,12.99,67.38,C,...,0,0,0,0,0,0,0,1,0,0
88093,88093,7082975,8744579,16425,16425,16375.0,60,12.99,373.64,B,...,0,0,0,1,0,0,0,0,0,0
217265,217265,1284896,1528668,8500,8500,8500.0,36,15.31,295.95,C,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136366,136366,5147014,6469325,12000,12000,11950.0,36,8.90,381.04,A,...,0,0,0,1,0,0,0,0,0,0
270195,270195,32359107,34972337,25000,25000,25000.0,60,20.99,676.20,E,...,0,0,0,1,0,0,0,0,0,0
131746,131746,5626330,7008599,14100,14100,14100.0,36,17.27,504.61,C,...,0,0,0,1,0,0,0,0,0,0
378475,378475,17754159,19886665,7000,7000,7000.0,36,8.39,220.62,A,...,0,0,0,0,0,0,0,0,0,1


In [13]:
# Here we select a limited set of input variables in a new dataframe.
inputs_train_with_ref_cat = loan_data_inputs_train.loc[: , ['grade:A',
'grade:B',
'grade:C',
'grade:D',
'grade:E',
'grade:F',
'grade:G',
'home_ownership:RENT',
'home_ownership:OWN',
'home_ownership:MORTGAGE',
'addr_state:ND_NE_IA_NV_FL_HI_AL',
'addr_state:NM_VA',
'addr_state:NY',
'addr_state:OK_TN_MO_LA_MD_NC',
'addr_state:CA',
'addr_state:UT_KY_AZ_NJ',
'addr_state:AR_MI_PA_OH_MN',
'addr_state:RI_MA_DE_SD_IN',
'addr_state:GA_WA_OR',
'addr_state:WI_MT',
'addr_state:TX',
'addr_state:IL_CT',
'addr_state:KS_SC_CO_VT_AK_MS',
'addr_state:WV_NH_WY_DC_ME_ID',
'verification_status:Not Verified',
'verification_status:Source Verified',
'verification_status:Verified',
'purpose:educ__sm_b__wedd__ren_en__mov__house',
'purpose:credit_card',
'purpose:debt_consolidation',
'purpose:oth__med__vacation',
'purpose:major_purch__car__home_impr',
'initial_list_status:f',
'initial_list_status:w',
'term:36',
'term:60',
'emp_length:0',
'emp_length:1',
'emp_length:2-4',
'emp_length:5-6',
'emp_length:7-9',
'emp_length:10',
'mths_since_issue_d:<38',
'mths_since_issue_d:38-39',
'mths_since_issue_d:40-41',
'mths_since_issue_d:42-48',
'mths_since_issue_d:49-52',
'mths_since_issue_d:53-64',
'mths_since_issue_d:65-84',
'mths_since_issue_d:>84',
'int_rate:<9.548',
'int_rate:9.548-12.025',
'int_rate:12.025-15.74',
'int_rate:15.74-20.281',
'int_rate:>20.281',
'mths_since_earliest_cr_line:<140',
'mths_since_earliest_cr_line:141-164',
'mths_since_earliest_cr_line:165-247',
'mths_since_earliest_cr_line:248-270',
'mths_since_earliest_cr_line:271-352',
'mths_since_earliest_cr_line:>352',
'delinq_2yrs:0',
'delinq_2yrs:1-3',
'delinq_2yrs:>=4',
'inq_last_6mths:0',
'inq_last_6mths:1-2',
'inq_last_6mths:3-6',
'inq_last_6mths:>6',
'open_acc:0',
'open_acc:1-3',
'open_acc:4-12',
'open_acc:13-17',
'open_acc:18-22',
'open_acc:23-25',
'open_acc:26-30',
'open_acc:>=31',
'pub_rec:0-2',
'pub_rec:3-4',
'pub_rec:>=5',
'total_acc:<=27',
'total_acc:28-51',
'total_acc:>=52',
'acc_now_delinq:0',
'acc_now_delinq:>=1',
'total_rev_hi_lim:<=5K',
'total_rev_hi_lim:5K-10K',
'total_rev_hi_lim:10K-20K',
'total_rev_hi_lim:20K-30K',
'total_rev_hi_lim:30K-40K',
'total_rev_hi_lim:40K-55K',
'total_rev_hi_lim:55K-95K',
'total_rev_hi_lim:>95K',
'annual_inc:<20K',
'annual_inc:20K-30K',
'annual_inc:30K-40K',
'annual_inc:40K-50K',
'annual_inc:50K-60K',
'annual_inc:60K-70K',
'annual_inc:70K-80K',
'annual_inc:80K-90K',
'annual_inc:90K-100K',
'annual_inc:100K-120K',
'annual_inc:120K-140K',
'annual_inc:>140K',
'dti:<=1.4',
'dti:1.4-3.5',
'dti:3.5-7.7',
'dti:7.7-10.5',
'dti:10.5-16.1',
'dti:16.1-20.3',
'dti:20.3-21.7',
'dti:21.7-22.4',
'dti:22.4-35',
'dti:>35',
'mths_since_last_delinq:Missing',
'mths_since_last_delinq:0-3',
'mths_since_last_delinq:4-30',
'mths_since_last_delinq:31-56',
'mths_since_last_delinq:>=57',
'mths_since_last_record:Missing',
'mths_since_last_record:0-2',
'mths_since_last_record:3-20',
'mths_since_last_record:21-31',
'mths_since_last_record:32-80',
'mths_since_last_record:81-86',
'mths_since_last_record:>86',
]]

In [14]:
inputs_train_with_ref_cat

,grade:A,grade:B,grade:C,grade:D,grade:E,grade:F,grade:G,home_ownership:RENT,home_ownership:OWN,home_ownership:MORTGAGE,...,mths_since_last_delinq:4-30,mths_since_last_delinq:31-56,mths_since_last_delinq:>=57,mths_since_last_record:Missing,mths_since_last_record:0-2,mths_since_last_record:3-20,mths_since_last_record:21-31,mths_since_last_record:32-80,mths_since_last_record:81-86,mths_since_last_record:>86
327896,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
237691,0,0,0,1,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
249332,0,0,1,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,1,0,0
88093,0,1,0,0,0,0,0,1,0,0,...,1,0,0,1,0,0,0,0,0,0
217265,0,0,1,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136366,1,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
270195,0,0,0,0,1,0,0,0,0,1,...,1,0,0,1,0,0,0,0,0,0
131746,0,0,1,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
378475,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1


In [15]:
# Here we store the names of the reference category dummy variables in a list.
ref_categories = ['grade:G',
'home_ownership:RENT',
'addr_state:ND_NE_IA_NV_FL_HI_AL',
'verification_status:Verified',
'purpose:educ__sm_b__wedd__ren_en__mov__house',
'initial_list_status:f',
'term:60',
'emp_length:0',
'mths_since_issue_d:>84',
'int_rate:>20.281',
'mths_since_earliest_cr_line:<140',
'delinq_2yrs:>=4',
'inq_last_6mths:>6',
'open_acc:0',
'pub_rec:0-2',
'total_acc:<=27',
'acc_now_delinq:0',
'total_rev_hi_lim:<=5K',
'annual_inc:<20K',
'dti:>35',
'mths_since_last_delinq:0-3',
'mths_since_last_record:0-2']

In [16]:
inputs_train = inputs_train_with_ref_cat.drop(ref_categories, axis = 1)
# From the dataframe with input variables, we drop the variables with variable names in the list with reference categories. 
inputs_train.head()

,grade:A,grade:B,grade:C,grade:D,grade:E,grade:F,home_ownership:OWN,home_ownership:MORTGAGE,addr_state:NM_VA,addr_state:NY,...,mths_since_last_delinq:Missing,mths_since_last_delinq:4-30,mths_since_last_delinq:31-56,mths_since_last_delinq:>=57,mths_since_last_record:Missing,mths_since_last_record:3-20,mths_since_last_record:21-31,mths_since_last_record:32-80,mths_since_last_record:81-86,mths_since_last_record:>86
327896,0,0,0,1,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,1,0
237691,0,0,0,1,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
249332,0,0,1,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,1,0,0
88093,0,1,0,0,0,0,0,0,0,1,...,0,1,0,0,1,0,0,0,0,0
217265,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0


In [17]:
inputs_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 373028 entries, 327896 to 171250
Columns: 104 entries, grade:A to mths_since_last_record:>86
dtypes: int64(104)
memory usage: 298.8 MB


In [18]:
loan_data_targets_train=loan_data_targets_train.astype('int64')

# PD Model Estimation

In [33]:
reg=LogisticRegression()
reg.fit(inputs_train,loan_data_targets_train)

C:\Users\HP\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [34]:
reg.intercept_

array([-1.22191535])

In [35]:
reg.coef_

array([[ 1.0197846 ,  0.92278553,  0.74245568,  0.5575971 ,  0.35984105,
         0.19681121,  0.09517843,  0.11170612,  0.06132793,  0.03912304,
         0.07803163,  0.07979899,  0.07361208,  0.15465389,  0.10776164,
         0.19294705,  0.2446717 ,  0.24546916,  0.28504405,  0.34643576,
         0.45437095,  0.10225661,  0.0027716 ,  0.34796403,  0.23336453,
         0.24079826,  0.28932091,  0.11176185,  0.09655206,  0.11923179,
         0.09644137,  0.06732466,  0.07610745,  0.13279002,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.51059134,  0.95952573,  0.51258212,  0.23997178,  0.05518489,
         0.02366825,  0.01217853, -0.01816461,  0.02531671,  0.06063979,
         0.01216939, -0.02703576,  0.77819909,  0.61281904,  0.40431135,
        -0.02994357, -0.10682118, -0.12492668, -0.13317772, -0.13376444,
        -0.17175383, -0.17985541, -0.04907565,  0.48582722, -0.03642359,
        -0.01376169,  0.15778836,  0.06615002,  0.0

In [36]:
feature_name=inputs_train.columns.values

In [37]:
summary_table=pd.DataFrame(columns=['feature_name'],data=feature_name)
summary_table['Coefficients']=np.transpose(reg.coef_)
summary_table

,feature_name,Coefficients
0,grade:A,1.019785
1,grade:B,0.922786
2,grade:C,0.742456
3,grade:D,0.557597
4,grade:E,0.359841
...,...,...
99,mths_since_last_record:3-20,0.518187
100,mths_since_last_record:21-31,0.534656
101,mths_since_last_record:32-80,0.723685
102,mths_since_last_record:81-86,0.395866


In [25]:
# P values for sklearn logistic regression.

# Class to display p-values for logistic regression in sklearn.

from sklearn import linear_model
import scipy.stats as stat

class LogisticRegression_with_p_values:
    
    def __init__(self,*args,**kwargs):#,**kwargs):
        self.model = linear_model.LogisticRegression(*args,**kwargs)#,**args)

    def fit(self,X,y):
        self.model.fit(X,y)
        
        #### Get p-values for the fitted model ####
        denom = (2.0 * (1.0 + np.cosh(self.model.decision_function(X))))
        denom = np.tile(denom,(X.shape[1],1)).T
        F_ij = np.dot((X / denom).T,X) ## Fisher Information Matrix
        Cramer_Rao = np.linalg.inv(F_ij) ## Inverse Information Matrix
        sigma_estimates = np.sqrt(np.diagonal(Cramer_Rao))
        z_scores = self.model.coef_[0] / sigma_estimates # z-score for eaach model coefficient
        p_values = [stat.norm.sf(abs(x)) * 2 for x in z_scores] ### two tailed test for p-values
        
        self.coef_ = self.model.coef_
        self.intercept_ = self.model.intercept_
        self.p_values = p_values

In [26]:
# reg = LogisticRegression_with_p_values()
# # We create an instance of an object from the newly created 'LogisticRegression_with_p_values()' class.

In [27]:
# reg.fit(inputs_train, loan_data_targets_train)
# # Estimates the coefficients of the object from the 'LogisticRegression' class
# # with inputs (independent variables) contained in the first dataframe
# # and targets (dependent variables) contained in the second dataframe.

In [38]:
# Chi2 distribution for feature selection

from sklearn.feature_selection import chi2
score,p_value=chi2(inputs_train,loan_data_targets_train)
summary_table['p_value']=p_value
summary_table.index=summary_table.index+1
summary_table.loc[0]=['Intercept',reg.intercept_[0],np.nan]
summary_table=summary_table.sort_index()
summary_table

,feature_name,Coefficients,p_value
0,Intercept,-1.221915,NaN
1,grade:A,1.019785,0.000000e+00
2,grade:B,0.922786,3.430222e-238
3,grade:C,0.742456,2.943491e-10
4,grade:D,0.557597,4.667503e-279
...,...,...,...
100,mths_since_last_record:3-20,0.518187,2.269164e-02
101,mths_since_last_record:21-31,0.534656,3.870474e-02
102,mths_since_last_record:32-80,0.723685,2.590504e-30
103,mths_since_last_record:81-86,0.395866,4.716456e-02


In [39]:
pd.options.display.max_rows=None

In [40]:
summary_table

,feature_name,Coefficients,p_value
0,Intercept,-1.221915,NaN
1,grade:A,1.019785,0.000000e+00
2,grade:B,0.922786,3.430222e-238
3,grade:C,0.742456,2.943491e-10
4,grade:D,0.557597,4.667503e-279
5,grade:E,0.359841,0.000000e+00
6,grade:F,0.196811,0.000000e+00
7,home_ownership:OWN,0.095178,4.253564e-01
8,home_ownership:MORTGAGE,0.111706,1.172181e-75
9,addr_state:NM_VA,0.061328,1.033896e-01


In [50]:
inputs_train.columns.values

array(['grade:A', 'grade:B', 'grade:C', 'grade:D', 'grade:E', 'grade:F',
       'home_ownership:OWN', 'home_ownership:MORTGAGE',
       'addr_state:NM_VA', 'addr_state:NY',
       'addr_state:OK_TN_MO_LA_MD_NC', 'addr_state:CA',
       'addr_state:UT_KY_AZ_NJ', 'addr_state:AR_MI_PA_OH_MN',
       'addr_state:RI_MA_DE_SD_IN', 'addr_state:GA_WA_OR',
       'addr_state:WI_MT', 'addr_state:TX', 'addr_state:IL_CT',
       'addr_state:KS_SC_CO_VT_AK_MS', 'addr_state:WV_NH_WY_DC_ME_ID',
       'verification_status:Not Verified',
       'verification_status:Source Verified', 'purpose:credit_card',
       'purpose:debt_consolidation', 'purpose:oth__med__vacation',
       'purpose:major_purch__car__home_impr', 'initial_list_status:w',
       'term:36', 'emp_length:1', 'emp_length:2-4', 'emp_length:5-6',
       'emp_length:7-9', 'emp_length:10', 'mths_since_issue_d:<38',
       'mths_since_issue_d:38-39', 'mths_since_issue_d:40-41',
       'mths_since_issue_d:42-48', 'mths_since_issue_d:49-52',
 

In [58]:
#open_acc, total_acc,pub_rec,total_rev_hi_lim,delinq_2yrs
# these 5 features have very less importasnce 
# so we can drop these


In [60]:
# We are going to remove some features, the coefficients for all or almost all of the dummy variables for which,
# are not tatistically significant or p_value > 0.05.

# We do that by specifying another list of dummy variables as reference categories, and a list of variables to remove.
# Then, we are going to drop the two datasets from the original list of dummy variables.

# Variables
inputs_train_with_ref_cat = loan_data_inputs_train.loc[: , ['grade:A',
'grade:B',
'grade:C',
'grade:D',
'grade:E',
'grade:F',
'grade:G',
'home_ownership:RENT',
'home_ownership:OWN',
'home_ownership:MORTGAGE',
'addr_state:ND_NE_IA_NV_FL_HI_AL',
'addr_state:NM_VA',
'addr_state:NY',
'addr_state:OK_TN_MO_LA_MD_NC',
'addr_state:CA',
'addr_state:UT_KY_AZ_NJ',
'addr_state:AR_MI_PA_OH_MN',
'addr_state:RI_MA_DE_SD_IN',
'addr_state:GA_WA_OR',
'addr_state:WI_MT',
'addr_state:TX',
'addr_state:IL_CT',
'addr_state:KS_SC_CO_VT_AK_MS',
'addr_state:WV_NH_WY_DC_ME_ID',
'verification_status:Not Verified',
'verification_status:Source Verified',
'verification_status:Verified',
'purpose:educ__sm_b__wedd__ren_en__mov__house',
'purpose:credit_card',
'purpose:debt_consolidation',
'purpose:oth__med__vacation',
'purpose:major_purch__car__home_impr',
'initial_list_status:f',
'initial_list_status:w',
'term:36',
'term:60',
'emp_length:0',
'emp_length:1',
'emp_length:2-4',
'emp_length:5-6',
'emp_length:7-9',
'emp_length:10',
'mths_since_issue_d:<38',
'mths_since_issue_d:38-39',
'mths_since_issue_d:40-41',
'mths_since_issue_d:42-48',
'mths_since_issue_d:49-52',
'mths_since_issue_d:53-64',
'mths_since_issue_d:65-84',
'mths_since_issue_d:>84',
'int_rate:<9.548',
'int_rate:9.548-12.025',
'int_rate:12.025-15.74',
'int_rate:15.74-20.281',
'int_rate:>20.281',
'mths_since_earliest_cr_line:<140',
'mths_since_earliest_cr_line:141-164',
'mths_since_earliest_cr_line:165-247',
'mths_since_earliest_cr_line:248-270',
'mths_since_earliest_cr_line:271-352',
'mths_since_earliest_cr_line:>352',
'inq_last_6mths:0',
'inq_last_6mths:1-2',
'inq_last_6mths:3-6',
'inq_last_6mths:>6',
'acc_now_delinq:0',
'acc_now_delinq:>=1',
'annual_inc:<20K',
'annual_inc:20K-30K',
'annual_inc:30K-40K',
'annual_inc:40K-50K',
'annual_inc:50K-60K',
'annual_inc:60K-70K',
'annual_inc:70K-80K',
'annual_inc:80K-90K',
'annual_inc:90K-100K',
'annual_inc:100K-120K',
'annual_inc:120K-140K',
'annual_inc:>140K',
'dti:<=1.4',
'dti:1.4-3.5',
'dti:3.5-7.7',
'dti:7.7-10.5',
'dti:10.5-16.1',
'dti:16.1-20.3',
'dti:20.3-21.7',
'dti:21.7-22.4',
'dti:22.4-35',
'dti:>35',
'mths_since_last_delinq:Missing',
'mths_since_last_delinq:0-3',
'mths_since_last_delinq:4-30',
'mths_since_last_delinq:31-56',
'mths_since_last_delinq:>=57',
'mths_since_last_record:Missing',
'mths_since_last_record:0-2',
'mths_since_last_record:3-20',
'mths_since_last_record:21-31',
'mths_since_last_record:32-80',
'mths_since_last_record:81-86',
'mths_since_last_record:>86',
]]

In [61]:
ref_categories = ['grade:G',
'home_ownership:RENT',
'addr_state:ND_NE_IA_NV_FL_HI_AL',
'verification_status:Verified',
'purpose:educ__sm_b__wedd__ren_en__mov__house',
'initial_list_status:f',
'term:60',
'emp_length:0',
'mths_since_issue_d:>84',
'int_rate:>20.281',
'mths_since_earliest_cr_line:<140',
'inq_last_6mths:>6',
'acc_now_delinq:0',
'annual_inc:<20K',
'dti:>35',
'mths_since_last_delinq:0-3',
'mths_since_last_record:0-2']

In [82]:
inputs_train2 = inputs_train_with_ref_cat.drop(ref_categories, axis = 1)
inputs_train2.head()

,grade:A,grade:B,grade:C,grade:D,grade:E,grade:F,home_ownership:OWN,home_ownership:MORTGAGE,addr_state:NM_VA,addr_state:NY,...,mths_since_last_delinq:Missing,mths_since_last_delinq:4-30,mths_since_last_delinq:31-56,mths_since_last_delinq:>=57,mths_since_last_record:Missing,mths_since_last_record:3-20,mths_since_last_record:21-31,mths_since_last_record:32-80,mths_since_last_record:81-86,mths_since_last_record:>86
327896,0,0,0,1,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,1,0
237691,0,0,0,1,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
249332,0,0,1,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,1,0,0
88093,0,1,0,0,0,0,0,0,0,1,...,0,1,0,0,1,0,0,0,0,0
217265,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0


In [84]:
feature_name = inputs_train2.columns.values
feature_name

array(['grade:A', 'grade:B', 'grade:C', 'grade:D', 'grade:E', 'grade:F',
       'home_ownership:OWN', 'home_ownership:MORTGAGE',
       'addr_state:NM_VA', 'addr_state:NY',
       'addr_state:OK_TN_MO_LA_MD_NC', 'addr_state:CA',
       'addr_state:UT_KY_AZ_NJ', 'addr_state:AR_MI_PA_OH_MN',
       'addr_state:RI_MA_DE_SD_IN', 'addr_state:GA_WA_OR',
       'addr_state:WI_MT', 'addr_state:TX', 'addr_state:IL_CT',
       'addr_state:KS_SC_CO_VT_AK_MS', 'addr_state:WV_NH_WY_DC_ME_ID',
       'verification_status:Not Verified',
       'verification_status:Source Verified', 'purpose:credit_card',
       'purpose:debt_consolidation', 'purpose:oth__med__vacation',
       'purpose:major_purch__car__home_impr', 'initial_list_status:w',
       'term:36', 'emp_length:1', 'emp_length:2-4', 'emp_length:5-6',
       'emp_length:7-9', 'emp_length:10', 'mths_since_issue_d:<38',
       'mths_since_issue_d:38-39', 'mths_since_issue_d:40-41',
       'mths_since_issue_d:42-48', 'mths_since_issue_d:49-52',
 

In [85]:
reg2 = LogisticRegression()
reg2.fit(inputs_train, loan_data_targets_train)

C:\Users\HP\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [67]:
# Same as above.
summary_table = pd.DataFrame(columns = ['Feature name'], data = feature_name)
summary_table['Coefficients'] = np.transpose(reg2.coef_)
summary_table.index = summary_table.index + 1
summary_table.loc[0] = ['Intercept', reg2.intercept_[0]]
summary_table = summary_table.sort_index()
summary_table

,Feature name,Coefficients
0,Intercept,-1.353597
1,grade:A,1.050039
2,grade:B,0.953428
3,grade:C,0.766366
4,grade:D,0.574605
...,...,...
80,mths_since_last_record:3-20,0.531544
81,mths_since_last_record:21-31,0.523750
82,mths_since_last_record:32-80,0.701796
83,mths_since_last_record:81-86,0.370488


In [86]:
pickle.dump(reg2, open('pd_model.sav', 'wb'))
# Here we export our model to a 'SAV' file with file name 'pd_model.sav'.